In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath('../'))


import numpy as np
import matplotlib.pyplot as plt

import astropy.units as u

import CtllDes 
from CtllDes.core import ctll, satellite

import poliastro 
from poliastro.bodies import Earth
from poliastro.twobody import Orbit
from poliastro.frames import Planes
from poliastro.constants import J2000

# Changing Propagator interval of integration
Once changed. It will be considered to be the exact solution the propagation within this timeframe. That is, all test will evaluate differences related to this propagation. 

In [ ]:
satellite.PROPAGATOR_DT = 1

In [38]:
#Orbit object, more info on poliastro API reference.
from poliastro.twobody import Orbit

#The bodies module specifies attractors
from poliastro.bodies import Earth


#Planes of reference and Epochs, are almost every time 
#the default ones, but just in case if you want to use 
#any other, they can be found in the following modules

#import planes of reference, EARTH_EQUATOR 
from poliastro.frames import Planes

#import J2000 time reference from constants 
from poliastro.constants import J2000

### Specify classical orbit parameters ###

a = 8000 * u.km # semi-major axis [distance]
ecc = 0 * u.one # eccentricity [dimensionleess]
inc = 98 * u.deg # inclination [angle] 
raan = 0 * u.rad # right ascencion of the ascending node [angle]
argp = 0 * u.rad # perigee argument [angle]
nu = 0 * u.rad # true anomaly [angle]

plane = Planes.EARTH_EQUATOR # not necessary to specify
epoch = J2000 # not necessary to specify

#classmethod of Orbit
orb = Orbit.from_classical(Earth,
                            a,
                            ecc,
                            inc,
                            raan,
                            argp,
                            nu)

# No Perturbations

In [39]:
sat = satellite.Sat.from_orbit(orb)

In [40]:
ref_r,rev_v = sat.rv(0.05,dt=5,drag=False,J2=False)

In [41]:
ref_ssps = sat.ssps(0.05,dt=5,drag=False,J2=False)

In [42]:
from IPython.display import clear_output
import time

propagator_dts = np.linspace(2,500,40)
rs = []
vs = []
times_r = []

for prop_dt in propagator_dts:
    clear_output(wait=True)
    satellite.PROPAGATOR_DT = prop_dt
    sat = satellite.Sat.from_orbit(orb)
    t = time.time()
    tmp_r, tmp_v = sat.rv(0.05, dt=5, drag=False, J2=False)
    times_r.append(time.time()-t)
    rs.append(tmp_r)
    vs.append(tmp_v)
    print(prop_dt)
    


500.0


In [43]:
ssps = []
times_ssps = []

for prop_dt in propagator_dts:
    clear_output(wait=True)
    satellite.PROPAGATOR_DT = prop_dt
    sat = satellite.Sat.from_orbit(orb)
    t = time.time()
    tmp_ssps = sat.ssps(0.05, dt=5, drag=False, J2=False)
    times_ssps.append(time.time()-t)
    ssps.append(tmp_ssps)
    print(prop_dt)
    

500.0


# Errors in r 

In [44]:
diff = []
for r in rs:
    diff.append(((np.sum(np.sqrt(np.sum((ref_r - r)**2,axis=1))))/r.shape[0]).value)


# Errors in (lat,lon) 

In [45]:
from CtllDes.utils import trigsf

In [46]:
diff_ssps = []
angles = []
for lonlat in ssps:
    tmp_angle = trigsf.get_angles(lonlat[0],lonlat[1],ref_ssps[0],ref_ssps[1])
    diff_ssps.append((np.nansum(tmp_angle.value))/(43200)) 

diff_ssps = np.array(diff_ssps)*Earth.R_mean.to(u.km)


In [61]:
import matplotlib.pyplot as plt
%matplotlib qt5

fig, ax = plt.subplots(nrows=2,ncols=1,figsize=(20,10))
ax[0].scatter(propagator_dts, diff,s=15,c='k',label="Non Perturbated Propagation")
ax[0].vlines(x=300,ymin=0,ymax=1.2, label="σ < 200 [m]",color='red')

ax[0].set_title("Trajectory deviation")
ax[0].set_ylabel("σ(dt) [km] ")
ax[0].set_xlabel("dt [s]")
ax[0].grid()
ax0t = ax[0].twinx()
ax0t.scatter(propagator_dts,times_r,c='orange',s=40,marker='+',label='time')
ax0t.set_ylabel("execution time [s]")
ax[0].legend(loc=6)

ax[1].scatter(propagator_dts, diff_ssps,s=15,c='k',label="Non Perturbated Propagation")
ax[1].vlines(x=300,ymin=0,ymax=0.0042, label="σₛₛₚ < 1 m",color='red')

ax[1].set_title("SSP deviation")
ax[1].set_ylabel("σₛₛₚ(dt) [km] ")
ax[1].set_xlabel("dt [s]")
ax[1].grid()
ax1t = ax[1].twinx()
ax1t.scatter(propagator_dts,times_ssps,c='orange',marker='+',s=20,label='time')
ax1t.set_ylabel("execution time [s]")
ax[1].legend(loc=6)
